# 05. PyTorch Going Modular

This section is all about turning jupyter notebook to python scripts

`Jupyter Notebook -> Python Scripts`

Going Modular is turning Notebook to series of different Python scripts that offer similar functionality.

for example, we can turn our notebook code into following Python files"
- `data_setup.py` - a file to prepare and download data if needed
- `engine.py` - a file containing various `training` functions
- `model_builder.py` - a file to create a PyTorch model
- `train.py` - a file to leverage all other files and train a target PyTorch model
- `utils.py` - a file dedicated to helpful utility functions